# Netflix Text Analysis

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from pprint import pprint
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

## Load Data

In [2]:
fp: Path = Path("../../../../data/NETFLIX_REVIEWS.parquet")

data: pl.DataFrame = pl.read_parquet(source=fp)
data.head(3)

,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
i64,str,str,str,str,i64,i64,str,str
0,"""7e73f80e-a8fd-…","""15261855397701…","""A Google user""","""Works great on…",5,1,"""1.2.0 build 81…","""2011-05-12 18:…"
1,"""dab55eca-c2a0-…","""23438294286543…","""A Google user""","""Works great on…",5,1,"""1.2.0 build 81…","""2011-05-12 18:…"
2,"""a3b8fa06-8b8f-…","""17447360460835…","""A Google user""","""Works great on…",5,0,"""1.5.2 build 38…","""2011-05-12 18:…"


In [3]:
data = data.drop(columns=["", "pseudo_author_id", "author_name", "review_likes"])
data.head(3)

review_id,review_text,review_rating,author_app_version,review_timestamp
str,str,i64,str,str
"""7e73f80e-a8fd-…","""Works great on…",5,"""1.2.0 build 81…","""2011-05-12 18:…"
"""dab55eca-c2a0-…","""Works great on…",5,"""1.2.0 build 81…","""2011-05-12 18:…"
"""a3b8fa06-8b8f-…","""Works great on…",5,"""1.5.2 build 38…","""2011-05-12 18:…"


In [4]:
# Check if GPU is available
device: str = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cpu

In [5]:
from transformers import pipeline


TASK: str = "text-classification"
OUTPUT_DIR: str = "distilbert-base-uncased-finetuned-netflix-ratings"
MODEL_CHECKPOINT: str = f"chineidu/{OUTPUT_DIR}"
netflix_classifier: pipeline = pipeline(task=TASK, model=MODEL_CHECKPOINT)

2023-12-17 21:38:28.986495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
id2label: dict[str, Any] = {
    1: "very poor",
    2: "poor",
    3: "okay",
    4: "very good",
    5: "excellent",
}
label2id: dict[str, Any] = {
    "very poor": 1,
    "poor": 2,
    "okay": 3,
    "very good": 4,
    "excellent": 5,
}


def get_sentiment(reviews: list[str]) -> list[list[str], list[float]]:
    """This is used to obtain the label and the corresponding score."""
    result: list[dict[str, Any]] = netflix_classifier(reviews)
    labels: list[str] = [x.get("label") for x in result]
    ratings: list[str] = [label2id.get(x.get("label")) for x in result]

    return (labels, ratings)

In [7]:
texts: list[str] = data.select(pl.col("review_text")).to_series().to_list()[10:15]
texts

['Works great on Nexus S, but it needs subtitles support!',
 "Works on 3g even tho quality isn't the best but still...might be my new favorite app!",
 'Working great on Nexus S, picked up right where I left it watching.',
 'Works great. So far so good on Wifi. Nexus 1.',
 "Well I've tried it out on my Evo 4G and it works. Tried it through WiFi, and the quality is pretty decent.; it's very snappy. Then I tried it through 3G and I was thinking the worst, and while the quality is diminished; it still works. I don't live in an area with great signal but it works really well, though the pixelation can get bad at times. Haven't tried it on 4G but I'm expecting great results."]

In [8]:
netflix_classifier(texts)

[{'label': 'very good', 'score': 0.7804526686668396},
 {'label': 'very good', 'score': 0.4879053831100464},
 {'label': 'excellent', 'score': 0.8773263692855835},
 {'label': 'excellent', 'score': 0.7898459434509277},
 {'label': 'very good', 'score': 0.8486072421073914}]

In [9]:
# Actual
data.select(pl.col("review_rating")).to_series().to_list()[10:15]

[4, 5, 5, 5, 4]

In [10]:
# Predicted
get_sentiment(reviews=texts)

(['very good', 'very good', 'excellent', 'excellent', 'very good'],
 [4, 4, 5, 5, 4])

In [11]:
from sentence_transformers import SentenceTransformer


ENCODER_CHECKPOINT: str = "sentence-transformers/all-MiniLM-L6-v2"
# Load the model from HuggingFace Hub
encoder = SentenceTransformer(ENCODER_CHECKPOINT, device=device)

encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [12]:
encoder.get_max_seq_length(), encoder.get_sentence_embedding_dimension()

(256, 384)

In [13]:
import os
from getpass import getpass

from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())  # read local .env file
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

RUN_CELL: bool = False

if RUN_CELL:
    # Get API key without displaying it
    QDRANT_API_KEY: str = getpass("Please enter your API key: ")

In [14]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.exceptions import UnexpectedResponse


URL: str = "http://localhost:6333"
client = QdrantClient(
    url=URL,
    api_key=QDRANT_API_KEY,
)
DIMENSION: int = encoder.get_sentence_embedding_dimension()
METRIC: Any = models.Distance.COSINE
COLLECTION_NAME: str = "netflix-sentiment-analysis"

# Create a collection
try:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
            distance=METRIC,
        ),
        optimizers_config=models.OptimizersConfigDiff(indexing_threshold=0),
        shard_number=2,
    )
# If collection already exists
except UnexpectedResponse as err:
    print(err)

Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `netflix-sentiment-analysis` already exists!"},"time":0.000012749}'

In [15]:
# Get collection details
collection_info: Any = client.get_collection(collection_name=COLLECTION_NAME)
print(collection_info)

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=775500,
    indexed_vectors_count=0,
    points_count=775500,
    segments_count=12,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=384,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None
            ),
            shard_number=2,
            sharding_method=None,
            replication_factor=1,
            write_consistency_factor=1,
            read_fan_out_factor=None,
            on_disk_payload=True,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=False,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=0,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None
    ),
    payload_schema={}
)

### Generate Embeddings

- Generate embeddings for all the reviews in the dataset.
- The embeddings, the sentiment label and score will be included in the Qdrant collection as metadata.
- The metadata will be used to understand the customer opinions.

In [16]:
print(data.head())

shape: (5, 5)
┌────────────────────┬────────────────────┬───────────────┬────────────────────┬───────────────────┐
│ review_id          ┆ review_text        ┆ review_rating ┆ author_app_version ┆ review_timestamp  │
│ ---                ┆ ---                ┆ ---           ┆ ---                ┆ ---               │
│ str                ┆ str                ┆ i64           ┆ str                ┆ str               │
╞════════════════════╪════════════════════╪═══════════════╪════════════════════╪═══════════════════╡
│ 7e73f80e-a8fd-4ff3 ┆ Works great on my  ┆ 5             ┆ 1.2.0 build        ┆ 2011-05-12        │
│ -b09b-502f0ad0…    ┆ Evo! Glad andr…    ┆               ┆ 819145-1.2.0-102   ┆ 18:50:37          │
│ dab55eca-c2a0-45a8 ┆ Works great on HTC ┆ 5             ┆ 1.2.0 build        ┆ 2011-05-12        │
│ -93e3-9860c1c5…    ┆ incredible. C…     ┆               ┆ 819145-1.2.0-102   ┆ 18:50:52          │
│ a3b8fa06-8b8f-4f2f ┆ Works great on     ┆ 5             ┆ 1.5.2 build 389    ┆ 2011-05-12        │
│ -a1fa-fd37c4cb…    ┆ nexus s            ┆               ┆                    ┆ 18:55:14          │
│ 837fdfa5-606d-4cec ┆ Working perfect    ┆ 5             ┆ 1.2.1 build        ┆ 2011-05-12        │
│ -9e9a-e4a83dad…    ┆ for me on EVO, r…  ┆               ┆ 843839-1.2.0-30    ┆ 19:31:46          │
│ a8aaecb2-6984-44f7 ┆ cm7 2.3.3 N1       ┆ 5             ┆ 1.5.2 build 389    ┆ 2011-05-12        │
│ -b958-3f89f64d…    ┆                    ┆               ┆                    ┆ 19:32:50          │
└────────────────────┴────────────────────┴───────────────┴────────────────────┴───────────────────┘

In [17]:
def retrieve_labels(ratings: list[int]) -> list[str]:
    """This is used to retrieve the labels."""
    labels: list[str] = [id2label.get(x) for x in ratings]

    return labels

In [18]:
retrieve_labels(ratings=[2, 3, 4, 5])

['poor', 'okay', 'very good', 'excellent']

In [19]:
data.select("review_timestamp").to_series().to_list()[0]

'2011-05-12 18:50:37'

In [20]:
from dateutil.parser import parse


def get_timestamp(dates: list[str]) -> list[float]:
    """This is used to convert the dates to timestamp."""
    timestamps: list[float] = [parse(d).timestamp() for d in dates]
    return timestamps

In [21]:
d: str = "2011-05-12 18:50:37"

get_timestamp(dates=[d])

[1305222637.0]

### Qdrant Upsert

- Using “Upsert” with Qdrant Vector Database [Blog](https://redandgreen.co.uk/qdrant-upsert/ai-ml/).
- [Official Documentation](https://qdrant.tech/documentation/concepts/points/#upload-points).


In [22]:
# Encode the entire data (Batch Upload)
import uuid

from tqdm.auto import tqdm


batch_size: int = 500
counter: int = 0

if RUN_CELL:
    for i in tqdm(range(0, len(data), batch_size)):

        # Find end of batch
        i_end: int = min(i + batch_size, data.shape[0])

        # Extract batch
        sample_rows: list[Any] = data.rows()[i:i_end]
        batch: pl.DataFrame = pl.DataFrame(
            sample_rows,
            schema=[
                "review_id",
                "review_text",
                "review_rating",
                "author_app_version",
                "review_date",  # renamed!
            ],
        )

        # IDs: list[str] = [str(uuid.uuid4()) for _ in range(batch.shape[0])]
        IDs: list[str] = batch.select(pl.col("review_id")).to_series().to_list()

        # Generate embeddings for batch
        batch_reviews: list[str] = (
            batch.select(pl.col("review_text")).to_series().to_list()
        )
        vectors: list[float] = encoder.encode(
            batch_reviews, show_progress_bar=False
        ).tolist()

        batch_ratings: list[str] = (
            batch.select(pl.col("review_rating")).to_series().to_list()
        )

        # Get sentiment label and score for reviews in the batch
        labels: list[str] = retrieve_labels(ratings=batch_ratings)

        # Add new column(s)
        batch: pl.DataFrame = pl.concat(
            [batch, pl.DataFrame({"label": labels})], how="horizontal"
        )
        dates: list[str] = batch.select(pl.col("review_date")).to_series().to_list()
        timestamp: list[float] = get_timestamp(dates=dates)
        timestamp_df: pl.DataFrame = pl.DataFrame(data={"timestamp": timestamp})
        batch: pl.DataFrame = pl.concat([batch, timestamp_df], how="horizontal")

        # Get metadata
        counter += batch_size
        if counter % 12_000 == 0:
            print(">>> [Creating payload] [Upserting data] <<<")
        payload_data: list[dict[str, Any]] = batch.to_dicts()

        # Upsert/insert these records to Qdrant
        _ = client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=IDs,
                payloads=payload_data,
                vectors=vectors,
            ),
        )

print(">>> [Done] <<<")

>>> [Done] <<<

### Note

- Due to the size of the data, I only upserted ~775.5k datapoints.
- I took ~1hr 58m to upsert.

<hr><br>

## Opinion Mining
- a
- b
- c

In [23]:
def count_sentiment(data: list[dict[str, Any]]) -> dict[str, int]:
    """This is used to count the sentiments.

    It has been modified for Qdrant Vector DB.
    """
    from collections import Counter

    sentiments: list[str] = [x.dict().get("payload").get("label") for x in data]
    result: dict[str, int] = dict(Counter(sentiments))

    return result

In [24]:
N: int = 5_000
query: str = "are the customers satisfied with the app?"
query_vector: list[float] = encoder.encode(query).tolist()


hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=None,
    limit=N,
)
# for hit in hits:
# print(hit.payload, "score:", hit.score)

print(hits[:5])

[
    ScoredPoint(
        id='bac9519b-0005-420d-970e-3ef515f1b3a5',
        version=369,
        score=0.84076256,
        payload={
            'author_app_version': '3.5.1 build 1566',
            'label': 'very good',
            'review_date': '2014-06-11 03:56:09',
            'review_id': 'bac9519b-0005-420d-970e-3ef515f1b3a5',
            'review_rating': 4,
            'review_text': 'This app is very nice. I mostly use it to entertain the kids when we are out and about,
but it has never given me any serious problems in that respect. From MY experiences ALONE, I can say I am a 
satisfied customers thus far.',
            'timestamp': 1402455369.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='af768543-c619-4486-b8b6-e6ae5581bafb',
        version=1202,
        score=0.723863,
        payload={
            'author_app_version': '5.16.1 build 26734',
            'label': 'poor',
            'review_date': '2018-06-19 06:09:06',
            'review_id': 'af768543-c619-4486-b8b6-e6ae5581bafb',
            'review_rating': 2,
            'review_text': 'Very good app, but totally unsatisfactory customer service. Please teach \nyour 
customer service reps to talk to customers with respect if we are \nhaving an issue that has to be resolved.',
            'timestamp': 1529384946.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='62c2478d-7ea7-4286-9614-efaa64b950cb',
        version=83,
        score=0.71924067,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2012-05-22 16:01:13',
            'review_id': '62c2478d-7ea7-4286-9614-efaa64b950cb',
            'review_rating': 5,
            'review_text': "I've been a customer for more than 2 years great app",
            'timestamp': 1337698873.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='513ebf2b-af2d-4436-9455-30e404ee530b',
        version=1399,
        score=0.7089151,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2019-03-03 19:44:08',
            'review_id': '513ebf2b-af2d-4436-9455-30e404ee530b',
            'review_rating': 5,
            'review_text': 'good app and customer service when needed',
            'timestamp': 1551638648.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='434e85d0-a6cc-4355-bc1a-3614f0574aa8',
        version=863,
        score=0.7002642,
        payload={
            'author_app_version': '4.13.2 build 14565',
            'label': 'excellent',
            'review_date': '2017-02-27 13:36:14',
            'review_id': '434e85d0-a6cc-4355-bc1a-3614f0574aa8',
            'review_rating': 5,
            'review_text': 'Have no complaints about app',
            'timestamp': 1488198974.0
        },
        vector=None,
        shard_key=None
    )
]

In [25]:
sentiment: dict[str, int] = count_sentiment(hits)
sentiment

{'very good': 684,
 'poor': 92,
 'excellent': 3767,
 'okay': 225,
 'very poor': 231,
 None: 1}

In [26]:
import plotly.express as px


def visualize_result(sentiment: dict[str, Any], query: str) -> Any:

    vis_data: pd.DataFrame = pd.DataFrame(sentiment, index=[0]).drop(columns=[None]).T
    vis_data = vis_data.reset_index(drop=False).rename(
        columns={0: "rating", "index": "label"}
    )
    fig = px.bar(
        vis_data,
        x="label",
        y="rating",
        labels=dict(index="rating", rating="count"),
        color="label",
        text_auto=True,
        title=query,
    )
    return fig.show()

In [27]:
visualize_result(sentiment=sentiment, query=query)

In [28]:
N: int = 5_000
query: str = "what are the important features?"
query_vector: list[float] = encoder.encode(query).tolist()


hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=None,
    limit=N,
)

print(hits[:15])

[
    ScoredPoint(
        id='52127cca-0b23-4d8d-91a4-cd1dcab5dae0',
        version=1341,
        score=0.6361048,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-12-11 23:27:51',
            'review_id': '52127cca-0b23-4d8d-91a4-cd1dcab5dae0',
            'review_rating': 5,
            'review_text': 'More details and features',
            'timestamp': 1544567271.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='506bb405-854f-45f5-870f-0004e6dd6bad',
        version=542,
        score=0.62138975,
        payload={
            'author_app_version': '3.13.0 build 5107',
            'label': 'excellent',
            'review_date': '2015-06-20 16:38:40',
            'review_id': '506bb405-854f-45f5-870f-0004e6dd6bad',
            'review_rating': 5,
            'review_text': 'All features',
            'timestamp': 1434814720.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='10c97cfa-5c0e-4cab-8f49-0180af97aa43',
        version=1017,
        score=0.61171573,
        payload={
            'author_app_version': '4.12.2 build 14444',
            'label': 'excellent',
            'review_date': '2017-08-22 20:51:25',
            'review_id': '10c97cfa-5c0e-4cab-8f49-0180af97aa43',
            'review_rating': 5,
            'review_text': 'The features are amazing',
            'timestamp': 1503431485.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='ccebc563-1c86-42c9-82b8-352a790c8a28',
        version=1518,
        score=0.5907492,
        payload={
            'author_app_version': '7.16.1 build 19 34329',
            'label': 'excellent',
            'review_date': '2019-07-09 16:32:22',
            'review_id': 'ccebc563-1c86-42c9-82b8-352a790c8a28',
            'review_rating': 5,
            'review_text': 'so many features!!',
            'timestamp': 1562686342.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='c577ea60-aebe-42cb-8d87-1d57af46baee',
        version=1078,
        score=0.58485156,
        payload={
            'author_app_version': '5.10.1 build 25262',
            'label': 'excellent',
            'review_date': '2017-12-03 08:00:42',
            'review_id': 'c577ea60-aebe-42cb-8d87-1d57af46baee',
            'review_rating': 5,
            'review_text': 'I like the features',
            'timestamp': 1512284442.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='63737fc2-1b78-49fb-81f4-4d38662d6eb0',
        version=1196,
        score=0.5743925,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-06-09 07:31:19',
            'review_id': '63737fc2-1b78-49fb-81f4-4d38662d6eb0',
            'review_rating': 5,
            'review_text': 'Cool features',
            'timestamp': 1528525879.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='ef060254-d134-404e-bee8-5ade36cc36f6',
        version=1284,
        score=0.56196004,
        payload={
            'author_app_version': '6.13.0 build 29940',
            'label': 'okay',
            'review_date': '2018-10-18 14:45:57',
            'review_id': 'ef060254-d134-404e-bee8-5ade36cc36f6',
            'review_rating': 3,
            'review_text': 'A few very obvious feature requirements missing',
            'timestamp': 1539870357.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='f6765295-cc76-4b28-a15d-2d6151a37b89',
        version=1180,
        score=0.5608344,
        payload={
            'author_app_version': '4.15.1 build 14947',
            'label': 'excellent',
            'review_date': '2018-05-16 14:12:46',
            'review_id': 'f6765295-cc76-4b28-a15d-2d6151a37b89',
            'review_rating': 5,

In [29]:
N: int = 5_000
query: str = "what do you like about it?"
query_vector: list[float] = encoder.encode(query).tolist()


hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=None,
    limit=N,
)

print(hits[:15])

[
    ScoredPoint(
        id='c3cb9b1b-0d69-433f-a94a-1602e77297a2',
        version=1515,
        score=0.75734,
        payload={
            'author_app_version': '6.15.0 build 31503',
            'label': 'excellent',
            'review_date': '2019-07-06 08:16:44',
            'review_id': 'c3cb9b1b-0d69-433f-a94a-1602e77297a2',
            'review_rating': 5,
            'review_text': 'I like everything about it!',
            'timestamp': 1562397404.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='41387562-b62c-4bda-a898-4726d22d3a73',
        version=931,
        score=0.7522397,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2017-05-12 00:50:17',
            'review_id': '41387562-b62c-4bda-a898-4726d22d3a73',
            'review_rating': 5,
            'review_text': 'I like everything about it',
            'timestamp': 1494546617.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='d529c1de-af28-4542-aa0c-dc0589c2ab76',
        version=1312,
        score=0.7522397,
        payload={
            'author_app_version': '6.11.0 build 29552',
            'label': 'excellent',
            'review_date': '2018-11-13 22:12:39',
            'review_id': 'd529c1de-af28-4542-aa0c-dc0589c2ab76',
            'review_rating': 5,
            'review_text': 'I like everything about it',
            'timestamp': 1542143559.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='79389e5d-150a-4d58-a438-6f08e7c22713',
        version=911,
        score=0.7522397,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2017-04-18 12:23:41',
            'review_id': '79389e5d-150a-4d58-a438-6f08e7c22713',
            'review_rating': 5,
            'review_text': 'I like everything about it',
            'timestamp': 1492514621.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='3a7dc4a9-afb0-4ad1-89dd-07c2352140c6',
        version=672,
        score=0.7522397,
        payload={
            'author_app_version': '4.4.1 build 7647',
            'label': 'excellent',
            'review_date': '2016-05-05 11:54:41',
            'review_id': '3a7dc4a9-afb0-4ad1-89dd-07c2352140c6',
            'review_rating': 5,
            'review_text': 'I like everything about it',
            'timestamp': 1462445681.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='a373718f-33ca-4d9a-bc08-530c9cb23aae',
        version=111,
        score=0.7522397,
        payload={
            'author_app_version': '1.8.1 build 573',
            'label': 'excellent',
            'review_date': '2012-09-07 21:50:49',
            'review_id': 'a373718f-33ca-4d9a-bc08-530c9cb23aae',
            'review_rating': 5,
            'review_text': 'I like everything about it',
            'timestamp': 1347051049.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='b167dc0f-1c1b-4a3e-9fae-3f57f34245ad',
        version=528,
        score=0.7522397,
        payload={
            'author_app_version': '',
            'label': 'very good',
            'review_date': '2015-05-16 08:56:38',
            'review_id': 'b167dc0f-1c1b-4a3e-9fae-3f57f34245ad',
            'review_rating': 4,
            'review_text': 'I like everything about it',
            'timestamp': 1431762998.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='2d09b707-9c58-4555-962f-234ab9a2ef9a',
        version=851,
        score=0.74062943,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2017-02-13 01:12:33',
            'review_id': '2d09b707-9c58-4555-962f-234ab9a2ef9a',
            'review_rating': 5,
            'review_te

In [30]:
visualize_result(sentiment=sentiment, query=query)

In [31]:
print(hits[-20:])

[
    ScoredPoint(
        id='d63eeef6-0830-4404-af3e-36a2fe004f8e',
        version=695,
        score=0.5598812,
        payload={
            'author_app_version': '4.5.1 build 7734',
            'label': 'excellent',
            'review_date': '2016-06-11 15:16:46',
            'review_id': 'd63eeef6-0830-4404-af3e-36a2fe004f8e',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1465654606.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='b57ce3d2-b3b7-47cd-a985-5629ff327457',
        version=872,
        score=0.5598812,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2017-03-10 02:09:37',
            'review_id': 'b57ce3d2-b3b7-47cd-a985-5629ff327457',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1489108177.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='6e630e51-e4d8-4a1a-a4b1-a3ced0a48091',
        version=683,
        score=0.5598812,
        payload={
            'author_app_version': '4.4.1 build 7647',
            'label': 'excellent',
            'review_date': '2016-05-28 15:17:06',
            'review_id': '6e630e51-e4d8-4a1a-a4b1-a3ced0a48091',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1464445026.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='4d944640-157c-424e-aa06-8d37d9913995',
        version=1308,
        score=0.5598812,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2018-11-10 18:26:42',
            'review_id': '4d944640-157c-424e-aa06-8d37d9913995',
            'review_rating': 5,
            'review_text': 'it is awesome',
            'timestamp': 1541870802.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='d6832d29-d51c-43f6-94ab-9d77e94ea30d',
        version=1029,
        score=0.5598812,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2017-09-13 17:06:50',
            'review_id': 'd6832d29-d51c-43f6-94ab-9d77e94ea30d',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1505318810.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='0c64ff15-87ba-4a16-8ec7-3f33abad26af',
        version=759,
        score=0.5598812,
        payload={
            'author_app_version': '',
            'label': 'excellent',
            'review_date': '2016-10-03 20:39:32',
            'review_id': '0c64ff15-87ba-4a16-8ec7-3f33abad26af',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1475523572.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='7887d089-e0d5-4d84-beb8-13e627b847ef',
        version=1011,
        score=0.5598812,
        payload={
            'author_app_version': '5.2.0 build 19086',
            'label': 'excellent',
            'review_date': '2017-08-13 04:51:06',
            'review_id': '7887d089-e0d5-4d84-beb8-13e627b847ef',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1502596266.0
        },
        vector=None,
        shard_key=None
    ),
    ScoredPoint(
        id='b70e6ba0-ad89-41be-ad08-6635dd3f9706',
        version=1189,
        score=0.5598812,
        payload={
            'author_app_version': '6.2.1 build 27557',
            'label': 'excellent',
            'review_date': '2018-05-29 19:52:46',
            'review_id': 'b70e6ba0-ad89-41be-ad08-6635dd3f9706',
            'review_rating': 5,
            'review_text': 'It is awesome',
            'timestamp': 1527619966.0
        },
        vector=None

In [35]:
# N: int = 3
# Generate timestamps for start and end time of the period
s_date, e_date = ["2015-10-16", "201-10-01"]  # yyyy-mm-dd
start_time: float = get_timestamp([s_date])[0]
end_time: float = get_timestamp([e_date])[0]

query_filter: dict[str, Any] = {
    "must": [
        {"key": "timestamp", "range": {"gte": start_time, "lte": end_time}},
    ]
}


query: str = "which update has the worst performance?"
query_vector: list[float] = encoder.encode(query).tolist()


hits: list[Any] = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    query_filter=query_filter,
    limit=N,
)
# for hit in hits:
#     print(hit.payload, "score:", hit.score)

print(len(hits))

ResponseHandlingException: timed out